In [3]:
import numpy as np
import cv2
import time

import imutils
from imutils.video import FileVideoStream, VideoStream
from utils import *

In [4]:
class Switch():
    
    def __init__(self, THRESH=200):
        self.THRESH = THRESH
        self.last_switch = time.time()
        self.setup()
        self.gap = 1 # gap btw two switch press
    
    
    def setup(self):      
        self.backgroundobject = cv2.createBackgroundSubtractorMOG2(detectShadows = False)

        fvs = FileVideoStream(path).start()
        time.sleep(2.0) #to allow web cam to open
        TIMER_SETUP = 2 # timer for capturing base image, get reading in posture
        t = time.time()

        while True:
            frame = get_frame(fvs)
            if frame is None:
                break
            curr = (time.time() - t)
            if curr > TIMER_SETUP:
                break
            cv2.putText(frame, str(int(TIMER_SETUP - curr)+1), POS_SCREEN, cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4)
            cv2.imshow("Setup", frame)
            cv2.waitKey(1)
        
        cv2.destroyAllWindows()
        
        cv2.putText(frame, 'Select Region for Switch', POS_SCREEN, cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_RED, 2)
        self.bbox = cv2.selectROI(frame, False) # bounding box
        cv2.destroyAllWindows()
    
    
    def update(self, frame): # returns True if Switch is pressed.
        x, y, w, h = self.bbox
        region = frame[y:y+h, x:x+w]
        fgmask = self.backgroundobject.apply(region)

        switch_thresh = np.sum(fgmask==255)
        cv2.putText(frame, str(switch_thresh), POS_SCREEN, cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_RED, 2)
        drawbox(self.bbox, frame) # shows the region of switch
        if time.time() - self.last_switch < self.gap:
            drawbox(self.bbox, frame, -1) # fills the rect when pressed (-1)
        if switch_thresh>self.THRESH and time.time() - self.last_switch > self.gap:  # to avoid continuous press (atleast small delay)
            drawbox(self.bbox, frame, -1)
            self.last_switch = time.time() # last switch time
            return True
        return False
    

In [9]:
switch = Switch(500)
bbox = switch.bbox

In [7]:
color = 3
def change():
    global color
    color = (color + 1) % 4

def colorFrame(frame):
    if color == 3: return 
    for i in range(3):
        if i == color: continue
        frame[:, :, i] = 0

In [11]:
fvs = FileVideoStream(path).start() #vs Video Stream
time.sleep(0.3) #to allow web cam to open

color = 3

while True:
    frame = get_frame(fvs)
    if frame is None:
        break
    orig = frame.copy()
    drawbox(bbox, orig)
    if switch.update(frame): # if pressed changes the color
        change()
        
    colorFrame(frame)
    cv2.imshow("Original", orig)
    cv2.imshow("Result", frame)
    
    k = cv2.waitKey(1) & 0xff
    if k == 13:
        break

cv2.destroyAllWindows()
fvs.stop()

In [ ]:
cv2.destroyAllWindows()
fvs.stop()